In [1]:
from pycaret.datasets import get_data
import pycaret.classification as clf

juice = get_data('juice')
session = clf.setup(data=juice, target='Purchase', silent=True, verbose=False)

topk = clf.compare_models(n_select=3, include=['rf', 'gbc', 'et'])
topk_tuned = [clf.tune_model(model) for model in topk]

bagger = [clf.ensemble_model(model, method='Bagging') for model in topk_tuned]
booster = [clf.ensemble_model(model, method='Boosting') for model in topk_tuned]
blender = clf.blend_models(topk_tuned)
stacker = clf.stack_models(topk_tuned)

best_automl = clf.automl(optimize='AUC')
best_automl = clf.finalize_model(best_automl)

best_model = clf.get_config('prep_pipe')
best_model.steps.append(['trained_model', best_automl])
print(">>", type(best_model.steps[-1][-1]))

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7867,0.8822,0.7333,0.7333,0.7333,0.5556,0.5556
1,0.8533,0.8970,0.8333,0.8065,0.8197,0.6961,0.6964
2,0.7200,0.8699,0.5172,0.6818,0.5882,0.3821,0.3905
3,0.8800,0.9415,0.7241,0.9545,0.8235,0.7352,0.7513
4,0.8133,0.9172,0.8621,0.7143,0.7813,0.6209,0.6293
5,0.8267,0.8666,0.6897,0.8333,0.7547,0.6225,0.6292
6,0.8400,0.9258,0.7586,0.8148,0.7857,0.6583,0.6594
7,0.8267,0.8921,0.6897,0.8333,0.7547,0.6225,0.6292
8,0.8514,0.9011,0.8276,0.8000,0.8136,0.6900,0.6903


>> <class 'sklearn.ensemble._stacking.StackingClassifier'>


In [3]:
best_model.steps[:-1]

[('dtypes',
  DataTypes_Auto_infer(categorical_features=[], display_types=False,
                       features_todrop=[], id_columns=['Id'],
                       ml_usecase='classification', numerical_features=[],
                       target='Purchase', time_features=[])),
 ('imputer',
  Simple_Imputer(categorical_strategy='not_available',
                 fill_value_categorical=None, fill_value_numerical=None,
                 numeric_strategy='mean', target_variable=None)),
 ('new_levels1',
  New_Catagorical_Levels_in_TestData(replacement_strategy='least frequent',
                                     target='Purchase')),
 ('ordinal', 'passthrough'),
 ('cardinality', 'passthrough'),
 ('znz', 'passthrough'),
 ('club_R_L', 'passthrough'),
 ('new_levels',
  New_Catagorical_Levels_in_TestData(replacement_strategy='least frequent',
                                     target='Purchase')),
 ('feature_time',
  Make_Time_Features(list_of_features=None,
                     time_feature

In [4]:
best_model.steps[-1]

['trained_model',
 StackingClassifier(cv=5,
                    estimators=[('gbc',
                                 GradientBoostingClassifier(ccp_alpha=0.0,
                                                            criterion='friedman_mse',
                                                            init=None,
                                                            learning_rate=0.1,
                                                            loss='deviance',
                                                            max_depth=7,
                                                            max_features=1.0,
                                                            max_leaf_nodes=None,
                                                            min_impurity_decrease=0.3,
                                                            min_impurity_split=None,
                                                            min_samples_leaf=4,
                                            